In [1]:
# Vector Rag with choroma db
#pip install chromadb==0.5.0

chunking

In [2]:
with open("damascus.txt","r") as file:
    fileContent = file.read()

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

textSplitter = RecursiveCharacterTextSplitter(
    chunk_size = 350,
    chunk_overlap = 75
)

chunks = textSplitter.split_text(fileContent)
len(chunks)

74

embedding

In [4]:
from uuid import uuid4
ids = [str(uuid4()) for _ in chunks]

In [5]:
import chromadb
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="vector_rag_damascus")


In [6]:
collection.add(
    documents=chunks,
    ids=ids
)

In [12]:
collectionData = collection.get()
collectionData.keys()

dict_keys(['ids', 'embeddings', 'metadatas', 'documents', 'uris', 'data'])

In [15]:
collectionData["documents"][2]

"Some government members and al-Assad family members tried to escape the encirclement. Maher al-Assad successfully fled to Iraq via helicopter, but the car of Bashar's cousins Ihab Makhlouf and Iyad Makhlouf was intercepted by rebels who killed Ihab and captured Iyad. The head of the palace security was still at his post at Assad's residence,"

model opening

In [16]:
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())
from langchain.chat_models import ChatOpenAI

In [ ]:
model = ChatOpenAI(
    model_name="gpt-4o-mini",
)


In [19]:
query = "What is the most recent date on which Damascus was captured by the rebels?"
model.predict(query)

"As of my last knowledge update in October 2023, Damascus has not been captured by rebel forces during the Syrian Civil War. While there have been significant battles and periods of intense fighting in and around the city, the Syrian government, backed by its allies, has retained control over Damascus. The conflict has seen various shifts in power throughout the country, but the capital has largely remained under government control. If you're looking for the most current developments, I recommend checking the latest news sources for updates beyond my last training cut-off."

RETRIEVAL

In [21]:
topk = 15
results = collection.query(
    query_texts=[query], 
    n_results=topk # 
)

results.keys()

dict_keys(['ids', 'distances', 'metadatas', 'embeddings', 'documents', 'uris', 'data'])

In [27]:
resultList = results["documents"][0]

AUGMENTED 

In [29]:
from langchain import PromptTemplate

ragTemplate = """"
###
{context}
###

###
RULES:
Learn from the CONTEXT and answer the following question.
###

QUESTION:
{question}
"""


ragPromt = PromptTemplate(
    input_variables=["context", "question"],
    template=ragTemplate,
)

In [30]:
promt = ragPromt.format(
    question=query,
    context=" ".join(data for data in resultList)
)

GENERATION

In [31]:
model.predict(promt)

'The most recent date on which Damascus was captured by the rebels is 8 December 2024.'